## 0. Install and Import Dependencies

In [ ]:
!pip install ibm_watson
!pip install ffmpeg moviepy

In [3]:
import subprocess
import moviepy.editor as mp
from ibm_watson import SpeechToTextV1
from ibm_watson.websocket import RecognizeCallback, AudioSource
from ibm_cloud_sdk_core.authenticators import IAMAuthenticator

## 1. Extract Audio

In [4]:
# Here we are calling the moviepy library to convert the video to mkv
my_clip = mp.VideoFileClip(r"videoml.mp4")
my_clip

In [5]:
my_clip.audio.write_audiofile(r"my_result.wav")

MoviePy - Writing audio in my_result.wav


MoviePy - Done.


## 2. Setup STT Service

In [6]:
# Navigate to https://cloud.ibm.com/catalog?category=ai#servicesv ---->text to speech

apikey = ''
url = ''

In [7]:
# Setup service
authenticator = IAMAuthenticator(apikey)
stt = SpeechToTextV1(authenticator=authenticator)
stt.set_service_url(url)

## 3. Open Audio Source and Convert

In [ ]:
with open(r'my_result.wav', 'rb') as f:
    res = stt.recognize(audio=f, content_type='audio/wav', model='en-AU_NarrowbandModel', continuous=True).get_result()

In [ ]:
res

## 4. Process Results and Output to Text

In [ ]:
len(res['results'])

In [ ]:
text = [result['alternatives'][0]['transcript'].rstrip() + '.\n' for result in res['results']]

In [ ]:
text = [para[0].title() + para[1:] for para in text]
transcript = ''.join(text)
with open('output.txt', 'w') as out:
    out.writelines(transcript)

In [ ]:
transcript